### Model Training

In [7]:
from src.myscripts.model import Conv1DClassifier
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import config

path=config.DATA_SPLIT_SAVE_DIR_PATH
x_train= np.load(os.path.join(path, "x_train.npy"), allow_pickle=True)
y_train= np.load(os.path.join(path, "y_train.npy"), allow_pickle=True)
x_val= np.load(os.path.join(path, "x_val.npy"), allow_pickle=True)
y_val= np.load(os.path.join(path, "y_val.npy"), allow_pickle=True)
x_train = x_train.astype(np.float32)
x_val = x_val.astype(np.float32)
y_train = y_train.astype("long")
y_val = y_val.astype("long")

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = x_train.shape[1:]  # (16000, 1)
model = Conv1DClassifier(num_classes=43, input_shape=input_shape).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [8]:
print(torch.cuda.is_available())

True


In [11]:
from src.myscripts.model import EarlyStopping

# Trenowanie
epochs = 50
early_stopping = EarlyStopping(patience=3, verbose=True)

# Listy do przechowywania strat
train_losses = []
val_losses = []

# Trenowanie modelu
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels)

    epoch_loss = running_loss / len(train_loader)
    acc = correct.item() / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {acc:.4f}")

    # Dodanie straty treningowej do listy
    train_losses.append(epoch_loss)

    # Ewaluacja na zbiorze walidacyjnym
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)

    val_loss /= len(val_loader)  # średnia strata na zbiorze walidacyjnym
    val_accuracy = correct.item() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Dodanie straty walidacyjnej do listy
    val_losses.append(val_loss)

    # Sprawdzamy, czy należy zatrzymać trening
    early_stopping(val_loss, model)

    # Jeśli early stopping mówi "stop", przerywamy trening
    if early_stopping.early_stop:
        print("Early stopping triggered. Stopping training.")
        break

# Ładujemy najlepszy model
model = early_stopping.load_best_model(model)

Epoch 1/50, Loss: 0.2366, Accuracy: 0.9330
Validation Loss: 0.1918, Validation Accuracy: 0.9862
Epoch 2/50, Loss: 0.2674, Accuracy: 0.9350
Validation Loss: 0.2435, Validation Accuracy: 0.9793
Validation loss did not improve for 1 epochs.
Epoch 3/50, Loss: 0.1765, Accuracy: 0.9369
Validation Loss: 0.1927, Validation Accuracy: 0.9793
Validation loss did not improve for 2 epochs.
Epoch 4/50, Loss: 0.1692, Accuracy: 0.9596
Validation Loss: 0.2275, Validation Accuracy: 0.9759
Validation loss did not improve for 3 epochs.
Epoch 5/50, Loss: 0.1414, Accuracy: 0.9586
Validation Loss: 0.2446, Validation Accuracy: 0.9793
Validation loss did not improve for 4 epochs.
Epoch 6/50, Loss: 0.1184, Accuracy: 0.9557
Validation Loss: 0.3551, Validation Accuracy: 0.9759
Validation loss did not improve for 5 epochs.
Epoch 7/50, Loss: 0.1513, Accuracy: 0.9557
Validation Loss: 0.2596, Validation Accuracy: 0.9724
Validation loss did not improve for 6 epochs.
Epoch 8/50, Loss: 0.1043, Accuracy: 0.9665
Validatio

KeyboardInterrupt: 

In [10]:
torch.save(model.state_dict(),f"{os.path.join(config.MODEL_PATH, type(model).__name__)}.pth")

AttributeError: module 'config' has no attribute 'MODEL_PATH'

In [6]:
np.save(f"C:/Users/Bcom_/Documents/Projekty/Rozpoznawanie_dzwiekow_gitarowych/data/model_history/{type(model).__name__}_val_losses.npy",val_losses)
np.save(f"C:/Users/Bcom_/Documents/Projekty/Rozpoznawanie_dzwiekow_gitarowych/data/model_history/{type(model).__name__}_train_losses.npy",train_losses)